copied from: https://medium.com/@geonextgis/getting-started-with-microsoft-planetary-computer-stac-api-67cbebe96e5e

In [1]:
import pystac_client
import planetary_computer
import odc.stac
import matplotlib.pyplot as plt
import geopandas as gpd
from pystac.extensions.eo import EOExtension as eo

In [2]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace
)


In [3]:
# from original example
# bbox_of_interest = [77.3521, 12.7235, 77.8477, 13.2186]
# time_of_interest = "2022-01-01/2022-12-31"

# FHL location on San Juans
bbox_of_interest = [-123.33711789011167, 48.30270109018668, -122.65866282644907, 48.93242519875351]

times_of_interest = [
    "2022-07-27/2022-07-27",
    "2022-08-04/2022-08-04",
    "2022-08-12/2022-08-12",
    "2022-08-20/2022-08-20",
    "2022-09-29/2022-09-29",
]

In [4]:
items_list = []
for time_stamp in times_of_interest:
    # Query the catalog and sort the filtered image collection based on 'cloud_cover' 
    search = catalog.search(
        collections=["landsat-c2-l2", "landsat-c1-l2"],
        bbox=bbox_of_interest,
        datetime=time_stamp,
        # query={"eo:cloud_cover": {"lt": 10}},
        # sortby=["eo:cloud_cover"]
    )

    items = search.item_collection()
    print(f"Returned {len(items)} Items")
    items_list.append(items)

Returned 2 Items
Returned 2 Items
Returned 3 Items
Returned 0 Items
Returned 2 Items


In [5]:
bands_of_interest = ["nir08", "red", "green", "blue", "qa_pixel", "lwir11"]

datas = []
for items in items_list:
    for selected_item in items:
        if all([
            band in selected_item.assets.keys() for band in bands_of_interest
        ]):
            data = odc.stac.stac_load(
                [selected_item], bands=bands_of_interest, bbox=bbox_of_interest
            ).isel(time=0)
            datas.append(data)

In [6]:
scale = 0.00341802
offset = 149.0

In [7]:
for d in datas:
    temperature = d["lwir11"].astype("float")
    temperature *= scale
    temperature += offset
    celsius = temperature - 273.15
    d['temperature_celsius'] = celsius
    fn = f"{str(d.time.values)}.nc" 
    print(fn)
    d.to_netcdf(fn)

2022-07-27T19:01:50.586279000.nc
2022-07-27T19:01:26.703735000.nc
2022-08-04T19:02:17.408501000.nc
2022-08-04T19:01:53.525933000.nc
2022-08-12T19:02:00.844506000.nc
2022-08-12T19:01:36.957727000.nc
2022-09-29T19:02:11.597592000.nc
2022-09-29T19:01:47.715049000.nc
